# Access cloud data examples tutorial

- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program and AWS Public Dataset Program
  
### Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org) -  [Twitter](https://twitter.com/ChelleGentemann)   - Farallon Institute
* [Dr. Ryan Abernathey](mailto:rpa@ldeo.columbia.edu) - [Twitter](https://twitter.com/rabernat) - LDEO
* [Henri Drake](mailto:hdrake@mit.edu) - [Twitter](https://twitter.com/henrifdrake) - MIT 

Credits: Tutorial review and comments.
* [Dr. Ed Armstrong](mailto:edward.m.armstrong@jpl.nasa.gov) - JPL PODAAC


### Data proximate computing: These are BIG datasets that you can analyze on the cloud without downloading the data. 


### Here we will demonstrate some ways to access the:
- Pangeo CMIP6 (climate models)
- Pangeo AVISO sea level and current data
- AWS MUR sea surface temperatures



### To run this notebook

Code is in the cells that have <span style="color: blue;">In [  ]:</span> to the left of the cell and have a colored background

To run the code:
- option 1) click anywhere in the cell, then hold shift and press Enter
- option 2) click on the Run button at the top of the page in the dashboard

### First start by importing libraries


In [ ]:
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 

import xarray as xr
import fsspec
from matplotlib import pyplot as plt
import numpy as np
import cartopy
import cartopy.crs as ccrs
import intake
import dask

xr.set_options(display_style="html")  #display dataset nicely 
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

## Read in AVISO sea surface height data using intake from the Pangeo datastore on Google Cloud


For this example we will use gridded [sea-surface altimetry data]("http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047) from The Copernicus Marine Environment:,

This is a widely used dataset in physical oceanography and climate.
    
![globe image](http://marine.copernicus.eu/documents/IMG/SEALEVEL_GLO_SLA_MAP_L4_REP_OBSERVATIONS_008_027.png)
    

The dataset has already been extracted from copernicus and stored in google cloud storage in [xarray-zarr](http://xarray.pydata.org/en/latest/io.html#zarr) format.



### Here we use the ``intake`` library to access the Pangeo datastore on google cloud

This call reads the metadata into memory but in a 'lazy' way.  You haven't actually touched the data yet which is why it is so fast.  Lazy loading means that no computation is performed until you actually ask values to be computed (more [here](http://xarray.pydata.org/en/stable/dask.html)).

In [ ]:
%%time
cat_pangeo = intake.open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml")
ds_aviso = cat_pangeo.ocean.sea_surface_height.to_dask()

# the number of GB involved in the reduction
print(ds_aviso.nbytes/1e9)

ds_aviso

### Plot the data

We are looking at the sea level anomaly in this plot and use ``.sel`` to select day to look at.  One of the nice features of ``xarray`` is that it allows you to use coordinates to select data in a way that makes it easy to understand later what you were trying to do.

``xarray`` plotting functions rely on matplotlib internally, but they make use of all available metadata to make the plotting operations more intuitive and interpretable. More plotting examples are given [here](http://xarray.pydata.org/en/stable/plotting.html)

Here we use [.plot](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.plot.html) method to create a figure of the data with several arguments to specify details in the plot.

In [ ]:
%%time
ds_aviso['sla'].sel(time='2015-01-01').plot(vmax=1, cmap='RdBu_r')

### Start a cluster, a group of computers that will work together.

(A cluster is the key to big data analysis on on Cloud.)

- This will set up a [dask kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html) cluster for your analysis and give you a path that you can paste into the top of the Dask dashboard to visualize parts of your cluster.  
- You don't need to paste the link below into the Dask dashboard for this to work, but it will help you visualize progress.
- Try 20 workers to start (during the tutorial) but you can increase to speed things up later

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

** ☝️ Don’t forget to click the link above or copy it to the Dask dashboard on the left to view the scheduler dashboard! **

### Plot a timeseries

#### Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

Xarray has a lot of nice build-in methods, such as [.resampe](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html#xarray-dataset-resample) which can upsample or downsample data and [.mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray-dataarray-mean). Here we use these to calculate monthly means.  We then take the monthly means and use ``.mean`` again, with the coordinates `latitude` and `longitude` as arguments to create a timeseries of the monthly mean.  Finally, we plot the data and label the axes.

In [ ]:
%%time
#create the data
sla_monthly = ds_aviso['sla'].resample(time='1MS').mean()

sla_monthly_timeseries = sla_monthly.mean({'latitude','longitude'})

#plot the data
sla_monthly_timeseries.plot(label='Full data')
plt.ylabel('Sea Level Anomaly [m]')
plt.title('Global Mean Sea Level')
plt.legend()
plt.grid()

## Read CMIP6 data from Google Cloud using intake

<img src="https://www.carbonbrief.org/wp-content/uploads/2019/11/cmip6_mips.jpg" alt="drawing" width=300>

The CMIP6 data is a huge collection of different experiements.  Access to these data uses the [intake-esm](https://github.com/NCAR/intake-esm) library which you then use the catalog to select specific variables, experiments, or activities. (**Note: intake-esm is quite news and experimental.**)  There are some great tutorials [here](https://github.com/hdrake/cmip6-temperature-demo/) and [here](https://github.com/pangeo-data/pangeo-cmip6-examples/).

More information on CMIP6 is [here](https://www.earthsystemcog.org/projects/wip/CMIP6DataRequest) and variable names [here](https://docs.google.com/document/d/1h0r8RZr_f3-8egBMMh7aqLwy3snpD6_MrDz1q8n5XUk/edit)

A nice introduction is [here](https://towardsdatascience.com/a-quick-introduction-to-cmip6-e017127a49d3)

The Pangeo catalog listing is [here](https://pangeo-data.github.io/pangeo-datastore/cmip6_pangeo.html)

In [ ]:
%%time
col = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")

col

## Search the collection for historical, monthly, air temperature, for one realization

You can use the `variable id` (link given above) to search for different parameters, change the `table id` from atmospheric monthly to ocean monthly, 3hrly data etc.  More information on what you can search for is in this [tutorial](https://intake-esm.readthedocs.io/en/latest/notebooks/tutorial.html)

In [ ]:
cat_cmip = col.search(experiment_id=['ssp585','historical'],  # pick the `historical` forcing experiment
                 table_id='Amon',             # choose to look at atmospheric variables (A) saved at monthly resolution (mon)
                 variable_id='tas',           # choose to look at near-surface air temperature (tas) as our variable
                 member_id = 'r1i1p1f1')      # arbitrarily pick one realization for each model (i.e. just one set of initial conditions)
cat_cmip

## Convert data catalog into a dictionary of xarray datasets

`dset_dict` is a dictionary of [xarray.Datasets](https://www.carbonbrief.org/wp-content/uploads/2019/11/cmip6_mips.jpg) where its keys refer to compatible groups.

We then set the time period we are interested in and create a new dictionary containing the variable of interest


In [ ]:
# supress some annoying warnings
import logging
urllib3_log = logging.getLogger("urllib3")
urllib3_log.setLevel(logging.CRITICAL)

dset_dict = cat_cmip.to_dataset_dict(zarr_kwargs={'consolidated': True})

time_slice = slice('1850','2015') # specific years that bracket our period of interest

ds_dict = {}

for name, ds in dset_dict.items():
    
    # rename spatial dimensions if necessary
    if ('longitude' in ds.dims) and ('latitude' in ds.dims):
        ds = ds.rename({'longitude':'lon', 'latitude': 'lat'}) 

    #make sure time index is unique, no repeated years....
    _, index = np.unique(ds['time'], return_index=True)
    ds = ds.isel(time=index)
        
    ds = ds.sel(time=time_slice) # subset the data for the time period of interest
    
    # drop redundant coordinates 
    for coord in ds.coords:
        if coord not in ['lat','lon','time']:
            ds = ds.drop(coord)
    
    # Add near-surface air temperature to dictionary
    ds_dict[name] = ds

## Look at how air temperatures have changed 

Look at the data we have, and create means for more recent and historical data to examine how air temperatures have changed.  The last step here is to set a data attribute so that when it is plotted the labels are correct.

In [ ]:
def compute_temp_diff(ds_cmip, name):
    try:
        now = ds_cmip.sel(time=slice('2000-01-01','2015-12-31')).mean('time')
        then = ds_cmip.sel(time=slice('1850-01-01','1950-12-31')).mean('time')
        return (now-then)['tas']
    except ValueError as ve:
        # some cmip models have weird calendars that cause the above step to fail
        print(name, ve)
        pass
    
all_temp_diffs = dask.compute({name: compute_temp_diff(ds_cmip, name)
                         for name, ds_cmip in ds_dict.items()})[0]
list(all_temp_diffs.keys())

In [ ]:
model = 'CMIP.NCAR.CESM2-FV2.historical.Amon.gn'
temperature_change = all_temp_diffs[model]
temperature_change.attrs['long_name'] = 'Change in air temperature (K)'
temperature_change

## Plot the change in temperature

Here we use the excellent [cartopy](https://scitools.org.uk/cartopy/docs/latest/) library to plot our data on a projection and add both coastlines and borders.

In [ ]:
ortho = ccrs.Orthographic(-90, 20)           # define target coordinate frame
geo = ccrs.PlateCarree()                     # define origin coordinate frame

plt.figure(figsize=(9,7))                    #set the figure size
ax = plt.subplot(1, 1, 1, projection=ortho)  #create the axis for plotting

q = temperature_change.plot(ax=ax, 
                            transform = geo, 
                            cmap='OrRd', 
                            vmin=0, 
                            vmax=3) # plot a colormap in transformed coordinates

ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-')
ax.set_title(f'Global Warming Air Temp - {model}',fontsize=16, ha='center');

#plt.savefig('./../../airtemp_warming_patterns.png',dpi=100,bbox_inches='tight')

## [MUR SST](https://podaac.jpl.nasa.gov/Multi-scale_Ultra-high_Resolution_MUR-SST) [AWS Public dataset program](https://registry.opendata.aws/mur/) 

### Access the MUR SST which is in an s3 bucket.  
### This Pangeo binder is running on Google Cloud and data access will be slower than running it on AWS.  

![image](https://podaac.jpl.nasa.gov/Podaac/thumbnails/MUR-JPL-L4-GLOB-v4.1.jpg)

This code is an example of how to read from a s3 bucket.  

Right now (2/16/2020) this takes ~1min on AWS and ~2 min on google cloud, there are two issues here and we are working to solve both.  
1. In our Zarr datastore the time coodinate is chunked.  We should have this fixed by 3/1/2020.
1. Some shortcomings in the s3fs and zarr formats have been identified.  To work on these, git issues were raised to the developers [here](https://github.com/dask/s3fs/issues/285) and [here](https://github.com/zarr-developers/zarr-python/issues/536)

In [ ]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
ds_sst

### Read entire 10 years of data at 1 point.

Select the ``analysed_sst`` variable over a specific time period, `lat`, and `lon` and load the data into memory.

In [ ]:
%%time
sst_timeseries = ds_sst['analysed_sst'].sel(time=slice('2010-01-01','2020-01-01'),
                                            lat=47,
                                            lon=-145
                                           ).load()
sst_timeseries.plot()

### The anomaly is more interesting...  

Use [.groupby](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html#xarray-dataarray-groupby) method to calculate the climatology and [.resample](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html#xarray-dataset-resample) method to then average it into 1-month bins

In [ ]:
sst_climatology = sst_timeseries.groupby('time.dayofyear').mean()
sst_anomaly = sst_timeseries.groupby('time.dayofyear')-sst_climatology
sst_anomaly_monthly = sst_anomaly.resample(time='1MS').mean()

#plot the data
sst_anomaly.plot()
sst_anomaly_monthly.plot()
plt.axhline(linewidth=2,color='k')

## Close the cluster

In [ ]:
client.close()
cluster.close()